In [ ]:
import sys
import pandas as pd
import numpy as np
import anndata
import scvi
import scanpy as sc
import matplotlib

seed = 10

scvi.settings.seed = 10

sc.logging.print_versions()

path_to_save = ".../o_kloetzer/Atlas/human_extension/scVI/A_Ext_II_No1"

In [ ]:
adata = sc.read(".../Atlas/Atlas_human_extension_II_3000HVG.h5ad")
adata

In [ ]:

scvi.model.SCVI.setup_anndata(
    adata,
    batch_key="proj",
    layer="counts", categorical_covariate_keys=["orig_ident"],
    continuous_covariate_keys=["percent_mt"])
model = scvi.model.SCVI(adata)
model
vae = scvi.model.SCVI(adata, n_layers=3, n_latent=30, gene_likelihood="nb", dropout_rate=0.1)
vae.train(max_epochs = 600, plan_kwargs={"lr":0.001}, early_stopping = True, early_stopping_patience = 15)
model = vae

In [ ]:
model.save(path_to_save, overwrite = True)

In [ ]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent
adata.layers["scvi_normalized"] = model.get_normalized_expression(
    library_size=10e4)

In [ ]:
sc.pp.neighbors(adata, n_pcs=30, use_rep="X_scVI", random_state=seed)
sc.tl.umap(adata, min_dist=0.3, random_state=seed)

In [ ]:
sc.pl.umap(adata, color=['proj'])

In [ ]:
adata.write(".../Atlas/Atlas_human_extension_II_3000HVG_integrated.h5ad")